In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

data_file = "/home/awias/data/Summerschool_2025/metadata_more.csv"

# Load your CSV file
df = pd.read_csv(data_file)

# Only keep the records where the substring 'train' is in filename_index
df = df[df['filename_index'].str.contains('train', na=False)].copy()

# Remove rows where taxonID_index is missing (since it's our target)
df = df.dropna(subset=['taxonID_index']).copy()

# Remove 'eventDate' column
df = df.drop(columns=['eventDate'], errors='ignore')

# Remove rows where at least two of the specified columns are NaN
cols_to_check = ['Habitat', 'Latitude', 'Longitude', 'Substrate']
df = df[df[cols_to_check].isnull().sum(axis=1) < 2].copy()

# Encode categorical variables
label_encoders = {}
categorical_features = ['Habitat', 'Substrate']

for col in categorical_features:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le
    
# Prepare final feature matrix (without date features)
feature_cols_final = ['Habitat_encoded', 'Latitude', 'Longitude', 'Substrate_encoded']

X = df[feature_cols_final]
y = df['taxonID_index']

# Encode target variable if it's categorical
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y.astype(str))

print(f"\nNumber of unique target classes: {len(target_encoder.classes_)}")
print(f"Target classes: {target_encoder.classes_[:10]}...")  # Show first 10

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Train XGBoost model with GPU support
print("\nTraining XGBoost model")
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    random_state=42,
    eval_metric='mlogloss' if len(np.unique(y_encoded)) > 2 else 'logloss',
    tree_method='gpu_hist',  # Use GPU for training with gpu_hist
    gpu_id=0,  # Use first GPU
    early_stopping_rounds=10
)

# Add validation set for early stopping
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

xgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val, y_val)],
    verbose=True
)

# Make predictions
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.3f}")

# Feature Importance Analysis
feature_importance = xgb_model.feature_importances_
feature_names = X.columns

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\nFeature Importance Ranking:")
print(importance_df)

# Plot feature importance
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df, y='feature', x='importance')
plt.title('XGBoost Feature Importance for Fungi Classification')
plt.xlabel('Feature Importance')
plt.tight_layout()
plt.show()

# More detailed analysis
print("\nDetailed Feature Analysis:")
for idx, row in importance_df.iterrows():
    feature = row['feature']
    importance = row['importance']
    print(f"{feature}: {importance:.4f}")
    
    # Show some statistics for this feature
    if feature in X.columns:
        feature_stats = X[feature].describe()
        print(f"  Min: {feature_stats['min']:.2f}, Max: {feature_stats['max']:.2f}, Mean: {feature_stats['mean']:.2f}")
    print()

# Create a correlation matrix for numerical features
numerical_features = ['Latitude', 'Longitude']
if len(numerical_features) > 1:
    plt.figure(figsize=(8, 6))
    correlation_matrix = X[numerical_features].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
    plt.title('Correlation Matrix of Numerical Features')
    plt.tight_layout()
    plt.show()

# Print decoded categorical mappings for interpretation
print("\nCategorical Feature Mappings:")
for col in categorical_features:
    le = label_encoders[col]
    mapping = dict(zip(range(len(le.classes_)), le.classes_))
    print(f"\n{col}:")
    for code, category in mapping.items():
        print(f"  {code}: {category}")

# Show target distribution
plt.figure(figsize=(12, 6))
target_counts = pd.Series(y_encoded).value_counts().sort_index()
plt.bar(range(len(target_counts)), target_counts.values)
plt.title('Distribution of Target Classes')
plt.xlabel('Encoded Target Class')
plt.ylabel('Count')
plt.xticks(range(0, len(target_counts), max(1, len(target_counts)//20)))
plt.tight_layout()
plt.show()

print(f"\nTarget distribution summary:")
print(f"Total classes: {len(target_counts)}")
print(f"Most common class frequency: {target_counts.max()}")
print(f"Least common class frequency: {target_counts.min()}")


Number of unique target classes: 181
Target classes: ['0.0' '1.0' '10.0' '100.0' '101.0' '102.0' '103.0' '104.0' '105.0'
 '106.0']...

Training set size: (2537, 4)
Test set size: (635, 4)

Training XGBoost model


/home/awias/anaconda3/envs/standard/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [14:21:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.starting_round = model.num_boosted_rounds()


[0]	validation_0-mlogloss:5.00084
[1]	validation_0-mlogloss:4.86021
[2]	validation_0-mlogloss:4.76831


KeyboardInterrupt: 